[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/Isotr0py/SakuraLLM-Notebooks/blob/main/Sakura-13B-Galgame-Kaggle-llama.cpp.ipynb)

In [ ]:
!git clone https://github.com/SakuraLLM/Sakura-13B-Galgame.git
%cd Sakura-13B-Galgame

!pip install "diskcache>=5.6.1"
!pip install llama-cpp-python -i https://sakurallm.github.io/llama-cpp-python/whl/cu121
!pip install -q -r requirements.llamacpp.txt
!pip install -q pyngrok

# install localtunnel
!npm install -g localtunnel

In [ ]:
# ngrokToken留空则使用localtunnel进行内网穿透
ngrokToken = ""
use_pinggy = True
MODEL = "sakura-14b-qwen2beta-v0.9.1-q6k"


from huggingface_hub import hf_hub_download
from pathlib import Path

if ngrokToken:
    from pyngrok import conf, ngrok
    conf.get_default().auth_token = ngrokToken
    conf.get_default().monitor_thread = False
    ssh_tunnels = ngrok.get_tunnels(conf.get_default())
    if len(ssh_tunnels) == 0:
        ssh_tunnel = ngrok.connect(5000)
        print('address：'+ssh_tunnel.public_url)
    else:
        print('address：'+ssh_tunnels[0].public_url)
else:
    import subprocess
    import threading
    def start_localtunnel(port):
        p = subprocess.Popen(["lt", "--port", f"{port}"], stdout=subprocess.PIPE)
        for line in p.stdout:
            print(line.decode(), end='')
    threading.Thread(target=start_localtunnel, daemon=True, args=(5000,)).start()

MODEL_PATH = f"./models/{MODEL}.gguf"
if not Path(MODEL_PATH).exists():
    hf_hub_download(repo_id="SakuraLLM/Sakura-14B-Qwen2beta-v0.9.1-GGUF", filename=f"{MODEL}.gguf", local_dir="models/")

!python server.py \
    --model_name_or_path $MODEL_PATH \
    --llama_cpp \
    --use_gpu \
    --model_version 0.9 \
    --trust_remote_code \
    --no-auth